In [1]:
import numpy as np
import pandas as pd
import glob
import os
import pickle as pkl

In [2]:
dirs = glob.glob('input/dataset/train/*')
dirs

['input/dataset/train/7',
 'input/dataset/train/3',
 'input/dataset/train/14',
 'input/dataset/train/2',
 'input/dataset/train/0',
 'input/dataset/train/11',
 'input/dataset/train/4',
 'input/dataset/train/8',
 'input/dataset/train/5',
 'input/dataset/train/1',
 'input/dataset/train/9',
 'input/dataset/train/6',
 'input/dataset/train/13',
 'input/dataset/train/10',
 'input/dataset/train/12']

In [3]:
images = list()
labels = list()
for d in dirs:
    images += glob.glob(d + '/*')
    labels += [int(os.path.basename(d))] * len(glob.glob(d + '/*'))

In [4]:
images[:10], labels[:10]

(['input/dataset/train/7/n04602956_3862_0.jpg',
  'input/dataset/train/7/n04197391_3278_0.jpg',
  'input/dataset/train/7/n04197391_10112_0.jpg',
  'input/dataset/train/7/n04197391_11085_1.jpg',
  'input/dataset/train/7/n04602956_3082_1.jpg',
  'input/dataset/train/7/n04602956_2792_0.jpg',
  'input/dataset/train/7/n04197391_13180_0.jpg',
  'input/dataset/train/7/n03238879_2797_0.jpg',
  'input/dataset/train/7/n03238879_13857_0.jpg',
  'input/dataset/train/7/n04602956_3101_0.jpg'],
 [7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [5]:
from sklearn.utils import shuffle
df = pd.DataFrame({'images': images, 'labels': labels})
df = shuffle(df)
df.head()

,images,labels
39383,input/dataset/train/1/n03980874_7556_0.jpg,1
30063,input/dataset/train/8/n03013438_14875_0.jpg,8
41224,input/dataset/train/9/n03226538_1699_0.jpg,9
22437,input/dataset/train/4/n03237992_29174_0.jpg,4
33227,input/dataset/train/5/n03978966_13362_0.jpg,5


In [6]:
df.shape

(62258, 2)

In [7]:
train = df.iloc[:52258, :]
validation = df.iloc[52258:, :]
train.shape, validation.shape

((52258, 2), (10000, 2))

In [8]:
batch_size = 128
target_vector_size = df['labels'].unique().shape

In [9]:
def train_generator():
    while True:
        for start in range(0, len(train), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(train))
            train_batch = train.iloc[start:end, :]
            for img_path, label in zip(train_batch['images'], train_batch['labels']):
                name = os.path.basename(img_path).split()[0]
                with open('input/dataset/train_features/{}.pkl'.format(name), 'rb') as file:
                    data = pkl.load(file)
                    x_batch.append(data[0])
                    y_batch.append(data[1])
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch
            
def valid_generator():
    while True:
        for start in range(0, len(validation), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(validation))
            validation_batch = validation.iloc[start:end, :]
            for img_path, label in zip(validation_batch['images'], validation_batch['labels']):
                name = os.path.basename(img_path).split()[0]
                with open('input/dataset/train_features/{}.pkl'.format(name), 'rb') as file:
                    data = pkl.load(file)
                    x_batch.append(data[0])
                    y_batch.append(data[1])
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch

In [10]:
for x, y in train_generator():
    print(x.shape, y.shape)
    break

(128, 7, 7, 512) (128, 15)


In [22]:
import tensorflow as tf

def f2_score(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 2 * precision * recall / (precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

In [27]:
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Input, Dropout, Flatten
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import SGD, RMSprop, Adam

input_features = Input(shape=(7, 7 , 512))
features = GlobalAveragePooling2D()(input_features)
hidden = Dense(4096, activation='sigmoid')(features)
hidden = Dropout(0.2)(hidden)
hidden = Dense(2048, activation='sigmoid')(hidden)
hidden = Dropout(0.5)(hidden)
# hidden = Dense(256, activation='relu')(hidden)
# hidden = Dropout(0.6)(hidden)
predictions = Dense(15, activation='softmax')(hidden)
    
model = Model(inputs=input_features, outputs=predictions)

sgd = SGD(lr=0.01, momentum=0.8)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=[f2_score])

callbacks = [EarlyStopping(monitor='val_f2_score',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4,
                               mode='min'),
             ModelCheckpoint(monitor='val_f2_score',
                             filepath='best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=False,
                             mode='max')]

In [28]:
model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(train)) / float(batch_size)),
                    epochs=50,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(validation)) / float(batch_size)))

Epoch 1/50
409/409 [==============================] - 6s 15ms/step - loss: 1.8560 - f2_score: 0.1603 - val_loss: 1.5225 - val_f2_score: 0.2550
Epoch 2/50
409/409 [==============================] - 6s 14ms/step - loss: 1.5458 - f2_score: 0.2762 - val_loss: 1.4449 - val_f2_score: 0.3139
Epoch 3/50
409/409 [==============================] - 6s 14ms/step - loss: 1.4737 - f2_score: 0.3114 - val_loss: 1.4102 - val_f2_score: 0.3409
Epoch 4/50
409/409 [==============================] - 6s 16ms/step - loss: 1.4317 - f2_score: 0.3350 - val_loss: 1.3841 - val_f2_score: 0.3572
Epoch 5/50
409/409 [==============================] - 6s 15ms/step - loss: 1.3997 - f2_score: 0.3509 - val_loss: 1.3659 - val_f2_score: 0.3671
Epoch 6/50
409/409 [==============================] - 6s 15ms/step - loss: 1.3775 - f2_score: 0.3624 - val_loss: 1.3511 - val_f2_score: 0.3798
Epoch 7/50
409/409 [==============================] - 6s 14ms/step - loss: 1.3556 - f2_score: 0.3743 - val_loss: 1.3398 - val_f2_score: 0.3859

In [29]:
model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(train)) / float(batch_size)),
                    epochs=20,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(validation)) / float(batch_size)))

Epoch 1/20
409/409 [==============================] - 6s 15ms/step - loss: 0.8878 - f2_score: 0.5999 - val_loss: 1.1837 - val_f2_score: 0.5212
Epoch 2/20
409/409 [==============================] - 6s 14ms/step - loss: 0.8841 - f2_score: 0.6020 - val_loss: 1.1831 - val_f2_score: 0.5219
Epoch 3/20
409/409 [==============================] - 6s 14ms/step - loss: 0.8813 - f2_score: 0.6033 - val_loss: 1.1843 - val_f2_score: 0.5224
Epoch 4/20
409/409 [==============================] - 6s 14ms/step - loss: 0.8781 - f2_score: 0.6043 - val_loss: 1.1837 - val_f2_score: 0.5228
Epoch 5/20
409/409 [==============================] - 6s 15ms/step - loss: 0.8767 - f2_score: 0.6066 - val_loss: 1.1849 - val_f2_score: 0.5236
Epoch 6/20
405/409 [============================>.] - ETA: 0s - loss: 0.8734 - f2_score: 0.6069
Epoch 00006: reducing learning rate to 9.999999310821295e-05.
409/409 [==============================] - 6s 15ms/step - loss: 0.8731 - f2_score: 0.6073 - val_loss: 1.1842 - val_f2_score: 0.

In [30]:
test_images = os.listdir('input/dataset/test/')

In [31]:
os.path.basename(test_images[5])

'n04489695_9899_0.jpg'

In [32]:
model.load_weights('best_weights.hdf5')

In [33]:
from tqdm import tqdm
image_name, category = list(), list()
for name in tqdm(test_images):
    name = name.split()[0]
    with open('input/dataset/test_features/{}.pkl'.format(name), 'rb') as file:
        features = pkl.load(file)
    
    image_name.append(name)
    label = np.argmax(model.predict(features))
    category.append(label)

100%|██████████| 21273/21273 [00:26<00:00, 791.88it/s]


In [34]:
res = pd.DataFrame({'image_name': image_name, 'category': category}, columns=['image_name', 'category'])
# res = res.append(res.iloc[:5412, :])
print(res.shape)
res.to_csv('submission.csv', index=False)

(21273, 2)


In [35]:
res.category.value_counts()

4     3446
3     2738
2     2655
9     2092
1     1944
6     1830
10    1644
12    1634
8     1137
13     990
11     421
7      310
5      172
14     136
0      124
Name: category, dtype: int64

In [36]:
model.load_weights('best_weights.hdf5')
model.evaluate_generator(generator=valid_generator(), 
                        steps=np.ceil(float(len(validation)) / float(batch_size)))

[1.1836065587997437, 0.524]

In [37]:
preds = model.predict_generator(generator=valid_generator(), 
                               steps=np.ceil(float(len(validation)) / float(batch_size)))

In [38]:
preds = np.argmax(preds, axis=1)
preds.shape

(10000,)

In [39]:
validation['labels'].value_counts()

4     1392
3     1309
2     1252
1      996
9      863
6      825
12     824
10     748
8      606
13     438
7      206
11     204
0      134
14     106
5       97
Name: labels, dtype: int64

In [40]:
pd.DataFrame({'preds': preds})['preds'].value_counts()

4     1581
3     1300
2     1283
9      985
1      917
6      884
12     806
10     745
8      521
13     431
11     202
7      138
5       72
14      69
0       66
Name: preds, dtype: int64